In [2]:
from argparse import Namespace
from pathlib import Path
from glob import glob
import json
import plotly.express as px

scenes = []
for path in glob("../output/*"):
    try:
        namespace = eval((Path(path) / "cfg_args").read_text())
        results = None
        if (Path(path) / "results.json").exists():
            results = json.loads((Path(path) / "results.json").read_text())
            scenes.append((namespace,results))
    except FileNotFoundError as e:
        print(e)

In [3]:
import pandas as pd

results_df = []
for namespace, results in scenes:
    print(namespace.source_path.split("/")[-1])
    if namespace.source_path.split("/")[-1] in ["ship","livingroom","mic","chair_background2",]:
        for name,res in results.items():
            name = name.replace("texture_per_gaussian","multitexture")
            scene_name = namespace.source_path.split("/")[-1]
            print(name)
            if len(name.split("_"))==3:
                num_render_images = 100 if scene_name=="livingroom" else 50
            else:
                num_render_images = int(name.split("_")[-1])
                name = "_".join(name.split("_")[:-1])
            print(name)
            r = {
                "scene_name": scene_name,
                "train_run": namespace.model_path.split("/")[-1],
                "train_mode": "depth" if "depth" in namespace.model_path else "normal",
                "render_mode": ("texture_per_gaussian" if "per" in name else "texture") if "texture" in name else "normal",
                "method_name":"_".join(name.split("_")[:-1]),
                "iteration":int(name.split("_")[-1]),
                "num_render_images": num_render_images,
                **res
            }
            if r["iteration"] in [2000, 3000, 4000, 5000, 7000, 10000, 13000, 16000, 20000, 25000, 30000, 35000, 40000, 47000, 55000, 70000]:
                results_df.append(r)


results_df = pd.DataFrame(results_df)

mic
normal_alpha_7000
normal_alpha_7000
multitexture_alpha_30000
multitexture_alpha_30000
multitexture_alpha_2000
multitexture_alpha_2000
normal_alpha_10000
normal_alpha_10000
multitexture_alpha_4000
multitexture_alpha_4000
normal_alpha_25000
normal_alpha_25000
texture_alpha_4000
texture_alpha_4000
texture_alpha_10000
texture_alpha_10000
texture_alpha_13000
texture_alpha_13000
texture_alpha_16000
texture_alpha_16000
normal_alpha_5000
normal_alpha_5000
multitexture_alpha_10000
multitexture_alpha_10000
texture_alpha_5000
texture_alpha_5000
multitexture_alpha_3000
multitexture_alpha_3000
texture_alpha_20000
texture_alpha_20000
multitexture_alpha_25000
multitexture_alpha_25000
texture_alpha_1000
texture_alpha_1000
multitexture_alpha_5000
multitexture_alpha_5000
normal_alpha_20000
normal_alpha_20000
texture_alpha_3000
texture_alpha_3000
normal_alpha_13000
normal_alpha_13000
texture_alpha_25000
texture_alpha_25000
multitexture_alpha_1000
multitexture_alpha_1000
normal_alpha_4000
normal_alpha

In [4]:
results_df["scene_name"].unique()

array(['mic', 'chair_background2', 'ship', 'livingroom'], dtype=object)

In [5]:
results_df = results_df.sort_values("num_gaussians")
results_df["run"] = results_df["train_mode"] + results_df["render_mode"]
is_texture = (results_df["render_mode"]=="texture")

results_df["memory_per_image"] = results_df["scene_name"].apply(lambda x: 151000 if x=="livingroom" else 312000)
results_df["num_training_images"] = results_df["scene_name"].apply(lambda x: 100 if x=="livingroom" else 50)
results_df["memory_needed"] = results_df["num_gaussians"]*(48*(~is_texture)+11)*4 + results_df["memory_per_image"]*results_df["num_training_images"]*is_texture

In [6]:
df = results_df[results_df["num_gaussians"]>=5000]
df = df[df["scene_name"]=="mic"]
df["run"].unique()

array(['depthtexture', 'normalnormal', 'normaltexture'], dtype=object)

In [7]:
# 

# def pareto_line(df,x,y,color,fn="max"):
#     df = df.copy()
#     if fn=="max":
#         df = df[df[y]==df.groupby(color)[y].cummax()]
#     else:
#         df = df[df[y]==df.groupby(color)[y].cummin()]
    
#     df = df[df["num_gaussians"] != df.groupby(color)["num_gaussians"].shift(-1)]
    
#     return px.line(df, x=x, y=y, color=color, log_x=True,markers=True)

# df = results_df[results_df["num_gaussians"]>=5000]
# #df = df[df["scene_name"]=="mic"]
# df = df[(df["run"]=="normalnormal") | (df["run"]=="depthtexture") | (df["run"]=="normaltexture") | (df["run"]=="normaltexture_per_gaussian") | (df["run"]=="depthtexture_per_gaussian")]

# display(df)

# fig = pareto_line(df,"num_gaussians","PSNR","run")
# fig.show()
# fig = pareto_line(df,"num_gaussians","LPIPS","run",fn="min")
# fig.show()
# fig = pareto_line(df,"num_gaussians","SSIM","run")
# fig.show()

In [8]:
# pareto_line(df,"memory_needed","PSNR","run").show()
# pareto_line(df,"memory_needed","LPIPS","run",fn="min").show()
# pareto_line(df,"memory_needed","SSIM","run").show()

In [9]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(results_df[(results_df["train_run"]=="ship_slow_depth")])

,scene_name,train_run,train_mode,render_mode,method_name,iteration,num_render_images,SSIM,PSNR,LPIPS,num_gaussians,run,memory_per_image,num_training_images,memory_needed
195,ship,ship_slow_depth,depth,texture,multitexture_alpha,2000,50,0.647448,21.509010,0.332145,3000,depthtexture,312000,50,15732000
193,ship,ship_slow_depth,depth,texture,texture_alpha,2000,50,0.657340,22.128166,0.280108,3000,depthtexture,312000,50,15732000
218,ship,ship_slow_depth,depth,texture,texture_alpha,2000,50,0.666160,21.645178,0.268527,3000,depthtexture,312000,50,15732000
209,ship,ship_slow_depth,depth,texture,multitexture_alpha,3000,50,0.665785,22.673340,0.307607,5733,depthtexture,312000,50,15852252
215,ship,ship_slow_depth,depth,texture,texture_alpha,4000,50,0.698561,23.512659,0.246406,5733,depthtexture,312000,50,15852252
213,ship,ship_slow_depth,depth,texture,texture_alpha,3000,50,0.703813,23.259089,0.238823,5733,depthtexture,312000,50,15852252
196,ship,ship_slow_depth,depth,texture,multitexture_alpha,4000,50,0.665235,22.696308,0.308765,5733,depthtexture,312000,50,15852252
200,ship,ship_slow_depth,depth,texture,texture_alpha,4000,50,0.707289,23.413555,0.237160,5733,depthtexture,312000,50,15852252
201,ship,ship_slow_depth,depth,texture,texture_alpha,3000,50,0.695638,23.422983,0.248080,5733,depthtexture,312000,50,15852252
212,ship,ship_slow_depth,depth,texture,multitexture_alpha,5000,50,0.700048,23.866165,0.269765,8641,depthtexture,312000,50,15980204


In [16]:
plot_data = []
df = results_df[results_df.groupby(["scene_name","run"])["PSNR"].cummax()==results_df["PSNR"]]
max_gaussians_list = (list(range(2000,10000,1000))+list(range(10000,100000,10000))+list(range(100000,600000,100000)))
for max_gaussians_last, max_gaussians in zip(max_gaussians_list[:-1],max_gaussians_list[1:]):
    ldf = df[(df["num_gaussians"]<=max_gaussians)]
    ldf = ldf.loc[ldf.groupby(["scene_name","run"])["PSNR"].idxmax()][["PSNR","LPIPS","SSIM","run","num_gaussians","memory_needed"]]
    ldf = ldf.groupby("run").mean().reset_index()
    # ldf = ldf.groupby("run").apply(lambda x: x.groupby("scene")[["PSNR","SSIM","LPIPS"]].max()).max().reset_index()
    ldf["num_gaussians"] = max_gaussians
    #print(ldf)
    plot_data.append(ldf)

plot_data = pd.concat(plot_data).reset_index(drop=True)
plot_data = plot_data.loc[list(plot_data.groupby("PSNR")["num_gaussians"].idxmin())]#+list(plot_data.groupby("run")["num_gaussians"].idxmax())]

plot_data = plot_data[plot_data["run"].apply(lambda x: x in ["normaltexture","depthtexture","normalnormal"])]
plot_data["run"] = plot_data["run"].apply(lambda x: {
    "normaltexture": "Textured Rendering",
    "depthtexture": "Textured Rendering + Depth Supervision",
    "depthtexture_per_gaussian": "Textured Rendering Multi Sample + Depth Supervision",
    "normaltexture_per_gaussian": "Textured Rendering Multi Sample",
    "normalnormal":"3DGS"
}[x])

# plot_data = plot_data[plot_data["run"].apply(lambda x: x in ["depthtexture_per_gaussian","depthtexture"])]
# plot_data["run"] = plot_data["run"].apply(lambda x: {
#     "depthtexture": "Single Sample Textured Rendering",
#     "depthtexture_per_gaussian": "Multi Sample Textured Rendering",
# }[x])

for x_name in ["num_gaussians","memory_needed"]:
    fig = px.line(
        plot_data[["PSNR","SSIM","LPIPS"]].stack().to_frame().reset_index(1).merge(plot_data,left_index=True,right_index=True),
        x=x_name,
        y=0,
        color="run",
        log_x=True,
        facet_col="level_1",
        facet_col_spacing=0.05,
        labels={
            "0": "Score",
            "num_gaussians": "Num. Primitives",
            "run": "Method"
        },
        markers=True
    )
    # import plotly.graph_objects as go
    # fig = go.Figure(data=fig.data + fig2.data)

    fig.update_yaxes(matches=None)
    fig.for_each_yaxis(lambda yaxis: yaxis.update(showticklabels=True))
    fig.update_layout(legend=dict(
        yanchor="middle",
        y=-0.4,
        xanchor="center",
        x=0.5
    ))
    fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
    fig.update_layout( autosize=False, width=1000, height=400)
    fig.write_image(f"../figures/compare_multisample_plot_{x_name}.pdf")
    fig.show()

In [11]:
df1 = plot_data[(plot_data["num_gaussians"]<50000)]
df1 = df1.loc[df1.groupby("run")["num_gaussians"].apply(lambda x: x.idxmax())]

df2 = plot_data[(plot_data["num_gaussians"]<500000)]
df2 = df2.loc[df2.groupby("run")["num_gaussians"].apply(lambda x: x.idxmax())]



In [12]:
# best_results = results_df.loc[results_df.groupby(["run","scene_name"])["PSNR"].idxmax()][
#     ["PSNR","SSIM","LPIPS","num_gaussians","memory_needed","train_mode","render_mode","run"]
# ].groupby(["run","train_mode","render_mode"]).mean().reset_index()
best_results = pd.concat([df1,df2])
best_results = df2
formatted_table = best_results[["PSNR","SSIM","LPIPS"]].applymap(lambda x: f"{x:.3f}" if x<1 else f"{x:.2f}")
formatted_table["#Primitives"] = best_results["num_gaussians"].apply(lambda x: f"{x//1000}k")
formatted_table["Memory"] = best_results["memory_needed"].apply(lambda x: f"{x//1000000} MB")
formatted_table["Method"] = best_results["run"]
#formatted_table["Method"] = best_results["run"].apply(lambda x: {"normaltexture": "Textured Rendering","depthtexture": "Textured Rendering + Depth Supervision", "normalnormal":"3DGS"}[x])
#formatted_table["Depth Supervision"] = best_results["train_mode"].apply(lambda x: {"depth": "Yes", "normal":"No"}[x])
formatted_table = formatted_table.loc[:,["Method","PSNR","SSIM","LPIPS","#Primitives","Memory"]] 
print(formatted_table.to_latex(index=False))
display(formatted_table)
# display(formatted_table.pivot(columns="Scene").swaplevel(0,1,"columns"))
# display(formatted_table.groupby(["Textured Rendering", "Depth Supervision"]).apply(lambda x: x.melt(id_vars="Scene")).pivot(columns="Scene"))

\begin{tabular}{llllll}
\toprule
Method & PSNR & SSIM & LPIPS & #Primitives & Memory \\
\midrule
3DGS & 31.40 & 0.880 & 0.203 & 403.0k & 95.0 MB \\
Textured Rendering & 25.26 & 0.742 & 0.190 & 229.0k & 25.0 MB \\
Textured Rendering + Depth Supervision & 30.06 & 0.889 & 0.100 & 44.0k & 17.0 MB \\
\bottomrule
\end{tabular}



,Method,PSNR,SSIM,LPIPS,#Primitives,Memory
59,3DGS,31.40,0.880,0.203,403.0k,95.0 MB
60,Textured Rendering,25.26,0.742,0.190,229.0k,25.0 MB
37,Textured Rendering + Depth Supervision,30.06,0.889,0.100,44.0k,17.0 MB


In [36]:
df = results_df[results_df["run"]=="depthtexture"]
df = df.loc[df.groupby(["num_render_images","scene_name"])["PSNR"].idxmax()]
df["num_render_images_percentage"] = df["num_render_images"]*100//df["num_training_images"]
df

,scene_name,train_run,train_mode,render_mode,method_name,iteration,num_render_images,SSIM,PSNR,LPIPS,num_gaussians,run,memory_per_image,num_training_images,memory_needed,num_render_images_percentage
87,chair_background2,chair_background2_slow_depth,depth,texture,texture_alpha,30000,1,0.321647,14.401400,0.506033,44079,depthtexture,312000,50,17539476,2
156,livingroom,livingroom_slow_depth,depth,texture,texture_alpha,70000,1,0.083009,7.922831,0.517448,61983,depthtexture,151000,100,17827252,1
48,mic,mic_slow_depth,depth,texture,texture_alpha,30000,1,0.375818,12.511668,0.495767,44888,depthtexture,312000,50,17575072,2
206,ship,ship_slow_depth,depth,texture,texture_alpha,30000,1,0.351770,12.994673,0.511480,36595,depthtexture,312000,50,17210180,2
83,chair_background2,chair_background2_slow_depth,depth,texture,texture_alpha,30000,5,0.541990,19.206701,0.348925,44079,depthtexture,312000,50,17539476,10
170,livingroom,livingroom_slow_depth,depth,texture,texture_alpha,70000,5,0.588843,16.488237,0.401976,61983,depthtexture,151000,100,17827252,5
44,mic,mic_slow_depth,depth,texture,texture_alpha,30000,5,0.621639,17.983904,0.285115,44888,depthtexture,312000,50,17575072,10
202,ship,ship_slow_depth,depth,texture,texture_alpha,30000,5,0.458854,15.986426,0.436222,36595,depthtexture,312000,50,17210180,10
106,chair_background2,chair_background2_slow_depth,depth,texture,texture_alpha,30000,10,0.774842,25.670654,0.151749,44079,depthtexture,312000,50,17539476,20
180,livingroom,livingroom_slow_depth,depth,texture,texture_alpha,70000,10,0.640255,18.358294,0.345906,61983,depthtexture,151000,100,17827252,10


In [41]:
import numpy as np
from PIL import Image

def get_percentage_invalid(scene_name, num_render_images, iteration):
    path = f"../output/{scene_name}_slow_depth/test/texture_alpha_{iteration}_{num_render_images}"
    x = [(np.array(Image.open(img_path))==0).mean() for img_path in glob(path+"/renders/*_before_blend.png")]
    return np.mean(x)
    
df["percentage_invalid"] = df.apply(lambda r: get_percentage_invalid(r["scene_name"],r["num_render_images"],r["iteration"]),axis=1)

../output/chair_background2_slow_depth/test/texture_alpha_30000_1
38
../output/livingroom_slow_depth/test/texture_alpha_70000_1
105
../output/mic_slow_depth/test/texture_alpha_30000_1
38
../output/ship_slow_depth/test/texture_alpha_30000_1
38
../output/chair_background2_slow_depth/test/texture_alpha_30000_5
38
../output/livingroom_slow_depth/test/texture_alpha_70000_5
105
../output/mic_slow_depth/test/texture_alpha_30000_5
38
../output/ship_slow_depth/test/texture_alpha_30000_5
38
../output/chair_background2_slow_depth/test/texture_alpha_30000_10
38
../output/livingroom_slow_depth/test/texture_alpha_70000_10
105
../output/mic_slow_depth/test/texture_alpha_30000_10
38
../output/ship_slow_depth/test/texture_alpha_30000_10
38
../output/chair_background2_slow_depth/test/texture_alpha_30000_20
38
../output/livingroom_slow_depth/test/texture_alpha_70000_20
105
../output/mic_slow_depth/test/texture_alpha_30000_20
38
../output/ship_slow_depth/test/texture_alpha_30000_20
38
../output/chair_back

In [43]:
#df = df.set_index(["scene_name","num_render_images_percentage"])
new_row = df.loc[(df["scene_name"]=="livingroom") & (df["num_render_images_percentage"]==70)].copy()
new_row["num_render_images_percentage"] = 80
df = pd.concat([df,new_row],axis=0)
new_row = df.loc[(df["scene_name"]=="livingroom") & (df["num_render_images_percentage"]==70)].copy()
new_row["num_render_images_percentage"] = 60
df = pd.concat([df,new_row],axis=0)
df = df.loc[(df["num_render_images_percentage"]!=70)]
df = df.loc[(df["num_render_images_percentage"]!=50)]
df = df.loc[(df["num_render_images_percentage"]!=30)]
df = df.sort_values("num_render_images_percentage")

In [47]:
df = df[df["num_render_images_percentage"]>5]
df.groupby("num_render_images_percentage")["PSNR"].mean()

num_render_images_percentage
10     17.883831
20     23.370454
40     26.533081
60     28.258373
80     28.890375
100    30.063229
Name: PSNR, dtype: float64

In [58]:
#px.line(,x="num_render_images_percentage",y="PSNR")

plot_data = df.groupby("num_render_images_percentage")[["PSNR","SSIM","LPIPS"]].mean().reset_index()
fig = px.line(
    plot_data[["PSNR","SSIM","LPIPS"]].stack().to_frame().reset_index(1).merge(plot_data,left_index=True,right_index=True),
    x="num_render_images_percentage",
    y=0,
    facet_col="level_1",
    facet_col_spacing=0.05,
    labels={
        "0": "Score",
        "num_gaussians": "Num. Primitives",
        "run": "Method",
        "num_render_images_percentage": "Num. Rendering Images (%)"
    },
    markers=True
)
# import plotly.graph_objects as go
# fig = go.Figure(data=fig.data + fig2.data)

fig.update_yaxes(matches=None)
fig.for_each_yaxis(lambda yaxis: yaxis.update(showticklabels=True))
fig.update_layout(legend=dict(
    yanchor="middle",
    y=-0.4,
    xanchor="center",
    x=0.5
))
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
fig.update_layout( autosize=False, width=1000, height=400)
fig.write_image(f"../figures/fewer_rendering_images.pdf")
fig.show()

In [60]:
fig = px.line(
    df.groupby("num_render_images_percentage")["percentage_invalid"].mean().reset_index(),
    x="num_render_images_percentage",
    y="percentage_invalid",
    labels={
        "num_render_images_percentage": "Num. Rendering Images (%)",
        "percentage_invalid": "Portion of Invalid Pixels",
    }
)
fig.update_layout( autosize=False, width=600, height=400)
fig.write_image(f"../figures/fewer_rendering_images_portion_invalid.pdf")
fig.show()